The goal of this code is to determine if there is any one feature in a play that can determine the likelyhood of a defense commiting pass interference. While running this expairiment it looks as though the position of the line of scrimage in relation to the end zone is the key feature in a defense getting called for pass interference. The interesting point in this data is that defensive pass interference is only called on 1.35% of plays in the 2018 season. So it appears that while a defensive team might want to look at limiting these penelties that lead to free yardage it appears that the call is not likly to happen and if it does it is more likly to be called closer to the end zone. With this data it could be interesting to see how the time of the penalty is and the score at the time the penalty is called. This could allow a defensive coach to determine if it is a disipline issue with the athlete or if the penalty is being called in a tight game in the fourth quarter when addrenaline is taking over for the athlete.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebr
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import os
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
 

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
games = pd.read_csv('../input/nfl-big-data-bowl-2021/games.csv')
players = pd.read_csv('../input/nfl-big-data-bowl-2021/players.csv')
plays = pd.read_csv('../input/nfl-big-data-bowl-2021/plays.csv')
total_weeks = pd.concat(map(pd.read_csv, ['/kaggle/input/nfl-big-data-bowl-2021/week1.csv', '/kaggle/input/nfl-big-data-bowl-2021/week2.csv'
                                    ,'/kaggle/input/nfl-big-data-bowl-2021/week3.csv', '/kaggle/input/nfl-big-data-bowl-2021/week4.csv'
                                    ,'/kaggle/input/nfl-big-data-bowl-2021/week5.csv','/kaggle/input/nfl-big-data-bowl-2021/week6.csv'
                                    ,'/kaggle/input/nfl-big-data-bowl-2021/week7.csv','/kaggle/input/nfl-big-data-bowl-2021/week8.csv'
                                    ,'/kaggle/input/nfl-big-data-bowl-2021/week9.csv','/kaggle/input/nfl-big-data-bowl-2021/week10.csv'
                                    ,'/kaggle/input/nfl-big-data-bowl-2021/week11.csv','/kaggle/input/nfl-big-data-bowl-2021/week12.csv'
                                    ,'/kaggle/input/nfl-big-data-bowl-2021/week13.csv', '/kaggle/input/nfl-big-data-bowl-2021/week14.csv'
                                    , '/kaggle/input/nfl-big-data-bowl-2021/week15.csv','/kaggle/input/nfl-big-data-bowl-2021/week16.csv'
                                    ,'/kaggle/input/nfl-big-data-bowl-2021/week17.csv']))
total_weeks = total_weeks.reset_index(drop = True)
# week1 = pd.read_csv('../input/nfl-big-data-bowl-2021/week1.csv')

In [ ]:
data = plays
data = data.fillna(130)

In [ ]:
data1 = data.drop(['gameId','playId','playDescription','possessionTeam','playType','personnelD','typeDropback','penaltyCodes','penaltyJerseyNumbers','passResult','offensePlayResult','playResult'
                  ,'yardlineSide','personnelO', 'offenseFormation', 'absoluteYardlineNumber'], axis=1)
for x in range(len(data1['gameClock'])):
    data1['gameClock'][x]= str(data1['gameClock'][x]).replace(':','')

['quarter', 'down', 'yardsToGo', 'yardlineNumber', 'defendersInTheBox','numberOfPassRushers', 'preSnapVisitorScore', 'preSnapHomeScore','gameClock', 'epa', 'isDefensivePI']

In [ ]:
X = data1.copy()
del X['isDefensivePI']

In [ ]:
y = [x for x in data1['isDefensivePI']]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
X, y, test_size=0.75)

In [ ]:
def RandomForest(x):
    rf = RandomForestClassifier(n_estimators = 100, criterion='entropy', max_features=x)
    rf.fit(X_train,y_train)
    return rf, rf.score(X_test,y_test)

In [ ]:
def features(rf):
    features = [[x] for x in rf[0].feature_importances_]
    [features[x].append(X.keys()[x]) for x in range(len(features))]
    features.sort()
    return [rf[1], features[-5:]]

In [ ]:
features(RandomForest(5))

In [ ]:
scores = []
for x in range(1,11):
    scores.append([x,features(RandomForest(x))])

In [ ]:
scores.sort(key=lambda x: x[1])
scores[9]

In [ ]:
num_pass_rush = [x for x in data1['epa']]
dpi = [x for x in data1['isDefensivePI']]
isDPI = []
NOTDPI = []
for x in range(len(dpi)):
    if dpi[x] == True:
        isDPI.append(num_pass_rush[x])
    else:
        NOTDPI.append(num_pass_rush[x])

In [ ]:
(len(isDPI)/(len(NOTDPI)+len(isDPI))) *100

In [ ]:
plt.hist(isDPI, bins=np.linspace(-1,7,8), edgecolor='black', linewidth=1.2, align = 'mid', label = 'DPI')
plt.xlabel('EPA')
plt.legend();

While defensive pass interference was only called on 1.35% of plays in the 2018 season there is a pattern when an offense is approacing 2 EPA. At the point between 1 and 2 is where the majority of pass interference calls are happening. It appears that once the offense is getting within field goal range the defense is more likly to either slip up or a referee is more likly to call defensive pass interfierence. Another data point that would be interesting to look at but was incomplete would be how the absolute yardline will afffect where players are getting called for pass interference. 